# COVID-19 Data Analysis - Part 1: Data Preparation

**Objective:** Load, clean, validate, and prepare datasets for comprehensive COVID-19 analysis

**Author:** Aaron George  
**Date:** 11/10/2025
**Dataset Period:** January 22, 2020 - July 27, 2020

---

## Overview

This notebook prepares two primary datasets for analysis:

1. **full_grouped.csv** - Daily COVID-19 data by country (time series)
2. **worldometer_data.csv** - Country data with population and testing information

**Why these two datasets?**
- `full_grouped`: Complete time series for trend analysis, country comparisons, growth rates
- `worldometer`: Population data for fair per-capita comparisons and testing analysis

---

## Table of Contents
1. [Import Libraries](#import)
2. [Load Raw Data](#load)
3. [Explore full_grouped Dataset](#explore-fg)
4. [Clean full_grouped Dataset](#clean-fg)
5. [Explore worldometer Dataset](#explore-wm)
6. [Clean worldometer Dataset](#clean-wm)
7. [Data Validation](#validate)
8. [Create Derived Columns](#derive)
9. [Final Summary](#summary)
10. [Save Cleaned Data](#save)

<a id='import'></a>
## 1. Import Libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# File operations
import os

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Configure plotly
pio.templates.default = "plotly_white"

print("Libraries imported successfully!")
print(f"   - Pandas version: {pd.__version__}")
print(f"   - NumPy version: {np.__version__}")

Libraries imported successfully!
   - Pandas version: 2.2.0
   - NumPy version: 1.26.4


<a id='load'></a>
## 2. Load Raw Data

Loading the two datasets we'll be using for this analysis.

In [2]:

# Load the two datasets
full_grouped = pd.read_csv('../data/raw/full_grouped.csv')
worldometer = pd.read_csv('../data/raw/worldometer_data.csv')

print("Datasets loaded successfully!\n")

# Display basic information

print("DATASET SHAPES\n")

print(f"1. full_grouped     : {full_grouped.shape[0]:>6,} rows × {full_grouped.shape[1]:>2} columns")
print(f"2. worldometer      : {worldometer.shape[0]:>6,} rows × {worldometer.shape[1]:>2} columns")
print("="*70)

Datasets loaded successfully!

DATASET SHAPES

1. full_grouped     : 35,156 rows × 10 columns
2. worldometer      :    209 rows × 16 columns


<a id='explore-fg'></a>
## 3. Explore full_grouped Dataset

Let's understand the structure, columns, data types, and quality of our primary time series dataset.

In [6]:
print("FULL_GROUPED DATASET - INITIAL EXPLORATION\n")


# Display basic info
print(f"\nShape: {full_grouped.shape}")
print(f"\nColumns ({len(full_grouped.columns)}):")
print(full_grouped.columns.tolist())

print(f"\nData Types:")
print(full_grouped.dtypes)

print(f"\nMissing Values:")
missing = full_grouped.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("No missing values found!")

print(f"\nDate Information:")
print(f"First date: {full_grouped['Date'].min()}")
print(f"Last date: {full_grouped['Date'].max()}")
print(f"Total unique dates: {full_grouped['Date'].nunique()}")

print(f"\nCountry Information:")
print(f"Total unique countries: {full_grouped['Country/Region'].nunique()}")

print(f"\nWHO Region Distribution:")
print(full_grouped['WHO Region'].value_counts())

print(f"\nFirst 5 Rows:")
display(full_grouped.head())

print(f"\nLast 5 Rows:")
display(full_grouped.tail())

print(f"\nStatistical Summary:")
display(full_grouped.describe())

FULL_GROUPED DATASET - INITIAL EXPLORATION


Shape: (35156, 10)

Columns (10):
['Date', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases', 'New deaths', 'New recovered', 'WHO Region']

Data Types:
Date              object
Country/Region    object
Confirmed          int64
Deaths             int64
Recovered          int64
Active             int64
New cases          int64
New deaths         int64
New recovered      int64
WHO Region        object
dtype: object

Missing Values:
No missing values found!

Date Information:
First date: 2020-01-22
Last date: 2020-07-27
Total unique dates: 188

Country Information:
Total unique countries: 187

WHO Region Distribution:
WHO Region
Europe                   10528
Africa                    9024
Americas                  6580
Eastern Mediterranean     4136
Western Pacific           3008
South-East Asia           1880
Name: count, dtype: int64

First 5 Rows:


,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0,0,0,0,0,0,Europe
2,2020-01-22,Algeria,0,0,0,0,0,0,0,Africa
3,2020-01-22,Andorra,0,0,0,0,0,0,0,Europe
4,2020-01-22,Angola,0,0,0,0,0,0,0,Africa



Last 5 Rows:


,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
35151,2020-07-27,West Bank and Gaza,10621,78,3752,6791,152,2,0,Eastern Mediterranean
35152,2020-07-27,Western Sahara,10,1,8,1,0,0,0,Africa
35153,2020-07-27,Yemen,1691,483,833,375,10,4,36,Eastern Mediterranean
35154,2020-07-27,Zambia,4552,140,2815,1597,71,1,465,Africa
35155,2020-07-27,Zimbabwe,2704,36,542,2126,192,2,24,Africa



Statistical Summary:


,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
count,3.515600e+04,35156.000000,3.515600e+04,3.515600e+04,35156.00000,35156.000000,35156.000000
mean,2.356663e+04,1234.068239,1.104813e+04,1.128443e+04,469.36375,18.603339,269.315593
std,1.499818e+05,7437.238354,6.454640e+04,8.997149e+04,3005.86754,115.706351,2068.063852
min,0.000000e+00,0.000000,0.000000e+00,-2.000000e+00,0.00000,-1918.000000,-16298.000000
25%,1.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000
50%,2.500000e+02,4.000000,3.300000e+01,8.500000e+01,2.00000,0.000000,0.000000
75%,3.640250e+03,78.250000,1.286250e+03,1.454000e+03,75.00000,1.000000,20.000000
max,4.290259e+06,148011.000000,1.846641e+06,2.816444e+06,77255.00000,3887.000000,140050.000000


<a id='clean-fg'></a>
## 4. Clean full_grouped Dataset

**Cleaning steps:**
1. Convert Date column to datetime format
2. Handle missing values in WHO Region
3. Ensure numeric columns are correct data type
4. Sort by Country and Date
5. Reset index
6. Verify data quality

In [7]:
# Create a copy for cleaning
full_grouped_clean = full_grouped.copy()


# Step 1: Convert Date to datetime

print("\n1. Converting Date column to datetime...")
full_grouped_clean['Date'] = pd.to_datetime(full_grouped_clean['Date'])
print(f"Date column type: {full_grouped_clean['Date'].dtype}")
print(f"Date range: {full_grouped_clean['Date'].min()} to {full_grouped_clean['Date'].max()}")


# Step 2: Handle Missing WHO Region

print("\n2. Checking for missing WHO Region values...")
missing_who = full_grouped_clean['WHO Region'].isnull().sum()
if missing_who > 0:
    print(f"Found {missing_who} missing WHO Region values")
    print(f"Filling with 'Unknown'")
    full_grouped_clean['WHO Region'].fillna('Unknown', inplace=True)
    print(f"Missing values filled")
else:
    print(f"No missing WHO Region values")


# Step 3: Ensure Numeric Columns are Correct Type

print("\n3. Validating numeric columns...")

numeric_columns = ['Confirmed', 'Deaths', 'Recovered', 'Active', 
                   'New cases', 'New deaths', 'New recovered']

for col in numeric_columns:
    if col in full_grouped_clean.columns:
        # Convert to numeric, coercing errors to NaN
        full_grouped_clean[col] = pd.to_numeric(full_grouped_clean[col], errors='coerce')
        # Fill any NaN with 0 (assuming missing means 0)
        full_grouped_clean[col].fillna(0, inplace=True)

print(f"All numeric columns validated")


# Step 4: Sort by Country and Date

print("\n4. Sorting by Country/Region and Date...")
full_grouped_clean = full_grouped_clean.sort_values(['Country/Region', 'Date'])
print(f"Data sorted")


# Step 5: Reset Index

print("\n5. Resetting index...")
full_grouped_clean = full_grouped_clean.reset_index(drop=True)
print(f"Index reset")


# Step 6: Verify Cleaning

print("\n6. Verification:")
print(f"Shape: {full_grouped_clean.shape}")
print(f"Date range: {full_grouped_clean['Date'].min().date()} to {full_grouped_clean['Date'].max().date()}")
print(f"Countries: {full_grouped_clean['Country/Region'].nunique()}")
print(f"Missing values: {full_grouped_clean.isnull().sum().sum()}")

print("\nfull_grouped_clean: CLEANING COMPLETE!")


1. Converting Date column to datetime...
Date column type: datetime64[ns]
Date range: 2020-01-22 00:00:00 to 2020-07-27 00:00:00

2. Checking for missing WHO Region values...
No missing WHO Region values

3. Validating numeric columns...
All numeric columns validated

4. Sorting by Country/Region and Date...
Data sorted

5. Resetting index...
Index reset

6. Verification:
Shape: (35156, 10)
Date range: 2020-01-22 to 2020-07-27
Countries: 187
Missing values: 0

full_grouped_clean: CLEANING COMPLETE!


### Quick Data Quality Check for full_grouped_clean

Let's verify the cleaned data looks correct:

In [8]:
print("DATA QUALITY CHECKS - full_grouped_clean")


# Check 1: No missing dates for any country
print("\n1. Checking date continuity...")
countries_with_gaps = []
for country in full_grouped_clean['Country/Region'].unique():
    country_data = full_grouped_clean[full_grouped_clean['Country/Region'] == country]
    expected_days = (country_data['Date'].max() - country_data['Date'].min()).days + 1
    actual_days = country_data['Date'].nunique()
    if expected_days != actual_days:
        countries_with_gaps.append(country)

if len(countries_with_gaps) == 0:
    print("All countries have continuous date records")
else:
    print(f"{len(countries_with_gaps)} countries have date gaps")
    print(f"(This may be expected if countries reported at different start dates)")

# Check 2: Cumulative numbers should not decrease
print("\n2. Checking if cumulative numbers ever decrease...")
issues = []
for country in full_grouped_clean['Country/Region'].unique()[:10]:  # Check first 10 as sample
    country_data = full_grouped_clean[full_grouped_clean['Country/Region'] == country].copy()
    country_data = country_data.sort_values('Date')
    
    # Check if Confirmed ever decreases
    if (country_data['Confirmed'].diff() < -100).any():  # -100 threshold for data corrections
        issues.append(f"{country}: Confirmed decreased")

if len(issues) == 0:
    print("Cumulative numbers are monotonically increasing (sample check)")
else:
    print(f"Found {len(issues)} potential issues:")
    for issue in issues[:5]:  # Show first 5
        print(f"{issue}")

# Check 3: Check for negative values
print("\n3. Checking for negative values...")
negative_cols = []
for col in ['Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases', 'New deaths', 'New recovered']:
    if (full_grouped_clean[col] < 0).any():
        negative_cols.append(col)

if len(negative_cols) == 0:
    print("No negative values found")
else:
    print(f"Negative values found in: {negative_cols}")

# Check 4: Sample a few countries
print("\n4. Sample data check - USA:")
usa_sample = full_grouped_clean[full_grouped_clean['Country/Region'] == 'US'].tail(3)
display(usa_sample[['Date', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases']])

print("\nQuality checks complete!")

DATA QUALITY CHECKS - full_grouped_clean

1. Checking date continuity...
All countries have continuous date records

2. Checking if cumulative numbers ever decrease...
Cumulative numbers are monotonically increasing (sample check)

3. Checking for negative values...
Negative values found in: ['Active', 'New deaths', 'New recovered']

4. Sample data check - USA:


,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases
32709,2020-07-25,US,4178970,146465,1279414,2753091,66439
32710,2020-07-26,US,4233923,146935,1297863,2789125,54953
32711,2020-07-27,US,4290259,148011,1325804,2816444,56336



Quality checks complete!


<a id='explore-wm'></a>
## 5. Explore worldometer Dataset

Now let's examine our second dataset which contains population and testing data.

In [9]:

print("WORLDOMETER DATASET - INITIAL EXPLORATION")


# Display basic info
print(f"\nShape: {worldometer.shape}")
print(f"\nColumns ({len(worldometer.columns)}):")
print(worldometer.columns.tolist())

print(f"\nData Types:")
print(worldometer.dtypes)

print(f"\nMissing Values:")
missing = worldometer.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("No missing values found!")

print(f"\nCountry Information:")
print(f"   Total countries: {len(worldometer)}")

print(f"\nContinent Distribution:")
if 'Continent' in worldometer.columns:
    print(worldometer['Continent'].value_counts())

print(f"\nFirst 5 Rows:")
display(worldometer.head())

print(f"\nStatistical Summary:")
display(worldometer.describe())

WORLDOMETER DATASET - INITIAL EXPLORATION

Shape: (209, 16)

Columns (16):
['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'WHO Region']

Data Types:
Country/Region       object
Continent            object
Population          float64
TotalCases            int64
NewCases            float64
TotalDeaths         float64
NewDeaths           float64
TotalRecovered      float64
NewRecovered        float64
ActiveCases         float64
Serious,Critical    float64
Tot Cases/1M pop    float64
Deaths/1M pop       float64
TotalTests          float64
Tests/1M pop        float64
WHO Region           object
dtype: object

Missing Values:
Continent             1
Population            1
NewCases            205
TotalDeaths          21
NewDeaths           206
TotalRecovered        4
NewRecovered        206
ActiveCases    

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.311981e+08,5032179,NaN,162804.0,NaN,2576668.0,NaN,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas
1,Brazil,South America,2.127107e+08,2917562,NaN,98644.0,NaN,2047660.0,NaN,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas
2,India,Asia,1.381345e+09,2025409,NaN,41638.0,NaN,1377384.0,NaN,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia
3,Russia,Europe,1.459409e+08,871894,NaN,14606.0,NaN,676357.0,NaN,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe
4,South Africa,Africa,5.938157e+07,538184,NaN,9604.0,NaN,387316.0,NaN,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa



Statistical Summary:


,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
count,2.080000e+02,2.090000e+02,4.000000,188.000000,3.000000,2.050000e+02,3.000000,2.050000e+02,122.000000,208.000000,187.000000,1.910000e+02,191.000000
mean,3.041549e+07,9.171850e+04,1980.500000,3792.590426,300.000000,5.887898e+04,1706.000000,2.766433e+04,534.393443,3196.024038,98.681176,1.402405e+06,83959.366492
std,1.047661e+08,4.325867e+05,3129.611424,15487.184877,451.199512,2.566984e+05,2154.779803,1.746327e+05,2047.518613,5191.986457,174.956862,5.553367e+06,152730.591240
min,8.010000e+02,1.000000e+01,20.000000,1.000000,1.000000,7.000000e+00,42.000000,0.000000e+00,1.000000,3.000000,0.080000,6.100000e+01,4.000000
25%,9.663140e+05,7.120000e+02,27.500000,22.000000,40.500000,3.340000e+02,489.000000,8.600000e+01,3.250000,282.000000,6.000000,2.575200e+04,8956.500000
50%,7.041972e+06,4.491000e+03,656.000000,113.000000,80.000000,2.178000e+03,936.000000,8.990000e+02,27.500000,1015.000000,29.000000,1.357020e+05,32585.000000
75%,2.575614e+07,3.689600e+04,2609.000000,786.000000,449.500000,2.055300e+04,2538.000000,7.124000e+03,160.250000,3841.750000,98.000000,7.576960e+05,92154.500000
max,1.381345e+09,5.032179e+06,6590.000000,162804.000000,819.000000,2.576668e+06,4140.000000,2.292707e+06,18296.000000,39922.000000,1238.000000,6.313960e+07,995282.000000


<a id='clean-wm'></a>
## 6. Clean worldometer Dataset

**Cleaning steps:**
1. Clean column names (remove spaces, slashes, special characters)
2. Identify and handle missing values appropriately
3. Ensure numeric columns are correct data type
4. Verify data quality

In [10]:
print("CLEANING worldometer DATASET")

# Create a copy for cleaning
worldometer_clean = worldometer.copy()


# Step 1: Clean Column Names

print("\n1. Cleaning column names...")
print(f"Original columns: {worldometer_clean.columns.tolist()[:5]}...")  # Show first 5

# Remove spaces, slashes, commas, and other special characters
worldometer_clean.columns = (
    worldometer_clean.columns
    .str.replace('/', '_', regex=False)
    .str.replace(' ', '_', regex=False)
    .str.replace(',', '', regex=False)
    .str.replace('.', '', regex=False)
    .str.strip()
)

print(f"Cleaned columns: {worldometer_clean.columns.tolist()[:5]}...")
print(f"Column names cleaned")


# Step 2: Handle Missing Values

print("\n2. Analyzing missing values...")

missing_analysis = worldometer_clean.isnull().sum()
missing_analysis = missing_analysis[missing_analysis > 0].sort_values(ascending=False)

if len(missing_analysis) > 0:
    print(f"\nColumns with missing values:")
    for col, count in missing_analysis.items():
        pct = (count / len(worldometer_clean)) * 100
        print(f"{col}: {count} ({pct:.1f}%)")
    
    # For numeric columns with missing values, we'll keep them as NaN
    # They represent "data not available" which is different from 0
    print(f"\nKeeping NaN for missing numeric values (represents unavailable data)")
    print(f"This is important for testing data which not all countries reported")
else:
    print("No missing values found")


# Step 3: Ensure Numeric Columns are Correct Type

print("\n3. Converting numeric columns...")

# Identify numeric columns (columns that should be numbers)
numeric_cols_wm = []
for col in worldometer_clean.columns:
    if col not in ['Country_Region', 'Continent', 'WHO_Region']:
        numeric_cols_wm.append(col)

# Convert to numeric
for col in numeric_cols_wm:
    worldometer_clean[col] = pd.to_numeric(worldometer_clean[col], errors='coerce')

print(f"{len(numeric_cols_wm)} numeric columns converted")


# Step 4: Verify Cleaning

print("\n4. Verification:")
print(f"Shape: {worldometer_clean.shape}")
print(f"Countries: {len(worldometer_clean)}")
print(f"Data types:")
print(worldometer_clean.dtypes.value_counts())

print("\nworldometer_clean: CLEANING COMPLETE!")

CLEANING worldometer DATASET

1. Cleaning column names...
Original columns: ['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases']...
Cleaned columns: ['Country_Region', 'Continent', 'Population', 'TotalCases', 'NewCases']...
Column names cleaned

2. Analyzing missing values...

Columns with missing values:
NewDeaths: 206 (98.6%)
NewRecovered: 206 (98.6%)
NewCases: 205 (98.1%)
SeriousCritical: 87 (41.6%)
WHO_Region: 25 (12.0%)
Deaths_1M_pop: 22 (10.5%)
TotalDeaths: 21 (10.0%)
TotalTests: 18 (8.6%)
Tests_1M_pop: 18 (8.6%)
TotalRecovered: 4 (1.9%)
ActiveCases: 4 (1.9%)
Continent: 1 (0.5%)
Population: 1 (0.5%)
Tot_Cases_1M_pop: 1 (0.5%)

Keeping NaN for missing numeric values (represents unavailable data)
This is important for testing data which not all countries reported

3. Converting numeric columns...
13 numeric columns converted

4. Verification:
Shape: (209, 16)
Countries: 209
Data types:
float64    12
object      3
int64       1
Name: count, dtype: int64

worldomet

### Quick Data Quality Check for worldometer_clean

Verify the cleaned worldometer data:

In [11]:
print("DATA QUALITY CHECKS - worldometer_clean")


# Check 1: Verify key columns exist
print("\n1. Checking for key columns...")
key_cols = ['Country_Region', 'Population', 'TotalCases', 'TotalDeaths', 
            'TotalRecovered', 'ActiveCases']
missing_cols = [col for col in key_cols if col not in worldometer_clean.columns]

if len(missing_cols) == 0:
    print("All key columns present")
else:
    print(f"Missing columns: {missing_cols}")

# Check 2: Population data availability
print("\n2. Checking population data...")
if 'Population' in worldometer_clean.columns:
    pop_available = worldometer_clean['Population'].notna().sum()
    pop_total = len(worldometer_clean)
    print(f"Population data: {pop_available}/{pop_total} countries ({pop_available/pop_total*100:.1f}%)")
else:
    print("No Population column found")

# Check 3: Testing data availability
print("\n3. Checking testing data...")
if 'TotalTests' in worldometer_clean.columns:
    tests_available = worldometer_clean['TotalTests'].notna().sum()
    tests_total = len(worldometer_clean)
    print(f"Testing data: {tests_available}/{tests_total} countries ({tests_available/tests_total*100:.1f}%)")
else:
    print("No TotalTests column found")

# Check 4: Check for negative values
print("\n4. Checking for negative values...")
numeric_cols_check = worldometer_clean.select_dtypes(include=[np.number]).columns
negative_found = False
for col in numeric_cols_check:
    if (worldometer_clean[col] < 0).any():
        negative_found = True
        print(f"Negative values in: {col}")

if not negative_found:
    print("No negative values found")

# Check 5: Sample data
print("\n5. Sample data check - Top 5 by cases:")
if 'TotalCases' in worldometer_clean.columns:
    sample = worldometer_clean.nlargest(5, 'TotalCases')[
        ['Country_Region', 'Population', 'TotalCases', 'TotalDeaths', 'TotalTests']
    ]
    display(sample)

print("\nQuality checks complete!")

DATA QUALITY CHECKS - worldometer_clean

1. Checking for key columns...
All key columns present

2. Checking population data...
Population data: 208/209 countries (99.5%)

3. Checking testing data...
Testing data: 191/209 countries (91.4%)

4. Checking for negative values...
No negative values found

5. Sample data check - Top 5 by cases:


,Country_Region,Population,TotalCases,TotalDeaths,TotalTests
0,USA,3.311981e+08,5032179,162804.0,63139605.0
1,Brazil,2.127107e+08,2917562,98644.0,13206188.0
2,India,1.381345e+09,2025409,41638.0,22149351.0
3,Russia,1.459409e+08,871894,14606.0,29716907.0
4,South Africa,5.938157e+07,538184,9604.0,3149807.0



Quality checks complete!


<a id='validate'></a>
## 7. Data Validation

Cross-checking both datasets for consistency and compatibility.

In [13]:
print("="*70)
print("CROSS-DATASET VALIDATION")
print("="*70)


# Check 1: Country Name Matching

print("\n1. Checking country name consistency between datasets...")

# Get latest date from full_grouped
latest_date = full_grouped_clean['Date'].max()
fg_countries = set(full_grouped_clean[full_grouped_clean['Date'] == latest_date]['Country/Region'].unique())
wm_countries = set(worldometer_clean['Country_Region'].unique())

print(f"full_grouped_clean countries: {len(fg_countries)}")
print(f"worldometer_clean countries: {len(wm_countries)}")

# Countries in full_grouped but not in worldometer
fg_only = fg_countries - wm_countries
if len(fg_only) > 0:
    print(f"\n   Countries in full_grouped but NOT in worldometer ({len(fg_only)}):")
    for country in sorted(list(fg_only))[:10]:  # Show first 10
        print(f"{country}")
    if len(fg_only) > 10:
        print(f"{len(fg_only)-10} more")

# Countries in worldometer but not in full_grouped
wm_only = wm_countries - fg_countries
if len(wm_only) > 0:
    print(f"\nCountries in worldometer but NOT in full_grouped ({len(wm_only)}):")
    for country in sorted(list(wm_only))[:10]:
        print(f"{country}")
    if len(wm_only) > 10:
        print(f"and {len(wm_only)-10} more")

# Common countries
common = fg_countries & wm_countries
print(f"\n Common countries in both datasets: {len(common)}")
print(f"We can merge data for these {len(common)} countries")


# Check 2: Verify Total Cases Match (approximately)

print("\n2. Verifying case numbers match between datasets...")

# Get latest data from full_grouped
fg_latest = full_grouped_clean[full_grouped_clean['Date'] == latest_date].copy()

# Sample comparison for common countries
sample_countries = ['US', 'Brazil', 'India', 'Russia', 'United Kingdom']
print(f"\n Sample comparison for {len(sample_countries)} countries:")
print(f"{'Country':<20} {'full_grouped':<15} {'worldometer':<15} {'Difference':<12}")
print(f"{'-'*62}")

for country in sample_countries:
    if country in fg_latest['Country/Region'].values:
        fg_cases = fg_latest[fg_latest['Country/Region'] == country]['Confirmed'].values[0]
        
        # Try to find in worldometer (might have different name)
        wm_row = worldometer_clean[worldometer_clean['Country_Region'].str.contains(country, case=False, na=False)]
        
        if len(wm_row) > 0:
            wm_cases = wm_row['TotalCases'].values[0] if pd.notna(wm_row['TotalCases'].values[0]) else 0
            diff = abs(fg_cases - wm_cases)
            diff_pct = (diff / fg_cases * 100) if fg_cases > 0 else 0
            print(f"   {country:<20} {fg_cases:>14,} {wm_cases:>14,} {diff_pct:>10.1f}%")

print(f"\nNote: Small differences are expected due to different data collection times")

print("\nCross-dataset validation complete!")

CROSS-DATASET VALIDATION

1. Checking country name consistency between datasets...
full_grouped_clean countries: 187
worldometer_clean countries: 209

   Countries in full_grouped but NOT in worldometer (16):
Brunei
Burma
Central African Republic
China
Congo (Brazzaville)
Congo (Kinshasa)
Cote d'Ivoire
Holy See
Kosovo
Saint Vincent and the Grenadines
6 more

Countries in worldometer but NOT in full_grouped (38):
Aruba
Bermuda
Brunei 
CAR
Caribbean Netherlands
Cayman Islands
Channel Islands
Congo
Curaçao
DRC
and 28 more

 Common countries in both datasets: 171
We can merge data for these 171 countries

2. Verifying case numbers match between datasets...

 Sample comparison for 5 countries:
Country              full_grouped    worldometer     Difference  
--------------------------------------------------------------
   US                        4,290,259      5,032,179       17.3%
   Brazil                    2,442,375      2,917,562       19.5%
   India                     1,480,073   

<a id='derive'></a>
## 8. Create Derived Columns

Adding calculated columns to enhance analysis capabilities.

### For full_grouped_clean:
- Daily growth rate (percentage change in confirmed cases)
- 7-day moving average of new cases
- 14-day moving average of new cases
- Mortality rate (deaths per 100 cases)
- Recovery rate (recovered per 100 cases)

### For worldometer_clean:
- Cases per million population (if not already present)
- Deaths per million population (if not already present)
- Test positivity rate (cases / tests * 100)
- Serious cases percentage (serious / active * 100)

In [17]:

# 1. Daily Growth Rate

print("\n1. Calculating daily growth rate...")

# Group by country and calculate percentage change
full_grouped_clean['Daily_Growth_Rate'] = (
    full_grouped_clean.groupby('Country/Region')['Confirmed']
    .pct_change() * 100
)

# Replace inf and very large values
full_grouped_clean['Daily_Growth_Rate'] = full_grouped_clean['Daily_Growth_Rate'].replace([np.inf, -np.inf], np.nan)
full_grouped_clean.loc[full_grouped_clean['Daily_Growth_Rate'] > 1000, 'Daily_Growth_Rate'] = np.nan

print("Daily growth rate calculated")


# 2. Moving Averages

print("\n2. Calculating moving averages...")

# 7-day moving average of new cases
full_grouped_clean['New_Cases_7MA'] = (
    full_grouped_clean.groupby('Country/Region')['New cases']
    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)

# 14-day moving average of new cases
full_grouped_clean['New_Cases_14MA'] = (
    full_grouped_clean.groupby('Country/Region')['New cases']
    .transform(lambda x: x.rolling(window=14, min_periods=1).mean())
)

# 7-day moving average of new deaths
full_grouped_clean['New_Deaths_7MA'] = (
    full_grouped_clean.groupby('Country/Region')['New deaths']
    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)

print("7-day and 14-day moving averages calculated")


# 3. Mortality and Recovery Rates

print("\n3. Calculating mortality and recovery rates...")

# Mortality rate (deaths per 100 cases)
full_grouped_clean['Mortality_Rate'] = (
    (full_grouped_clean['Deaths'] / full_grouped_clean['Confirmed']) * 100
).round(2)

# Recovery rate (recovered per 100 cases)
full_grouped_clean['Recovery_Rate'] = (
    (full_grouped_clean['Recovered'] / full_grouped_clean['Confirmed']) * 100
).round(2)

# Replace inf with NaN (happens when Confirmed = 0)
full_grouped_clean['Mortality_Rate'] = full_grouped_clean['Mortality_Rate'].replace([np.inf, -np.inf], np.nan)
full_grouped_clean['Recovery_Rate'] = full_grouped_clean['Recovery_Rate'].replace([np.inf, -np.inf], np.nan)

print("Mortality and recovery rates calculated")


# 4. Days Since First Case

print("\n4. Calculating days since first case...")

# Find first case date for each country
first_case_dates = (
    full_grouped_clean[full_grouped_clean['Confirmed'] > 0]
    .groupby('Country/Region')['Date']
    .min()
    .reset_index()
    .rename(columns={'Date': 'First_Case_Date'})
)

# Merge back
full_grouped_clean = full_grouped_clean.merge(first_case_dates, on='Country/Region', how='left')

# Calculate days since first case
full_grouped_clean['Days_Since_First_Case'] = (
    (full_grouped_clean['Date'] - full_grouped_clean['First_Case_Date']).dt.days
)

# Drop the helper column
full_grouped_clean = full_grouped_clean.drop('First_Case_Date', axis=1)

print("Days since first case calculated")


# Summary

print("\nSummary of derived columns added:")
derived_cols = ['Daily_Growth_Rate', 'New_Cases_7MA', 'New_Cases_14MA', 
                'New_Deaths_7MA', 'Mortality_Rate', 'Recovery_Rate', 
                'Days_Since_First_Case']
for col in derived_cols:
    print(f"{col}")

print(f"\nTotal columns now: {len(full_grouped_clean.columns)}")

print("\nDerived columns for full_grouped_clean: COMPLETE!")


1. Calculating daily growth rate...
Daily growth rate calculated

2. Calculating moving averages...
7-day and 14-day moving averages calculated

3. Calculating mortality and recovery rates...
Mortality and recovery rates calculated

4. Calculating days since first case...
Days since first case calculated

Summary of derived columns added:
Daily_Growth_Rate
New_Cases_7MA
New_Cases_14MA
New_Deaths_7MA
Mortality_Rate
Recovery_Rate
Days_Since_First_Case

Total columns now: 17

Derived columns for full_grouped_clean: COMPLETE!


In [18]:


# 1. Cases Per Million (if not already present or recalculate)

print("\n1. Calculating cases per million population...")

if 'Population' in worldometer_clean.columns and 'TotalCases' in worldometer_clean.columns:
    worldometer_clean['Cases_Per_Million_Calculated'] = (
        (worldometer_clean['TotalCases'] / worldometer_clean['Population']) * 1_000_000
    ).round(2)
    print("Cases per million calculated")
else:
    print("Cannot calculate - missing required columns")


# 2. Deaths Per Million (if not already present or recalculate)

print("\n2. Calculating deaths per million population...")

if 'Population' in worldometer_clean.columns and 'TotalDeaths' in worldometer_clean.columns:
    worldometer_clean['Deaths_Per_Million_Calculated'] = (
        (worldometer_clean['TotalDeaths'] / worldometer_clean['Population']) * 1_000_000
    ).round(2)
    print("Deaths per million calculated")
else:
    print("Cannot calculate - missing required columns")


# 3. Test Positivity Rate

print("\n3. Calculating test positivity rate...")

if 'TotalTests' in worldometer_clean.columns and 'TotalCases' in worldometer_clean.columns:
    worldometer_clean['Test_Positivity_Rate'] = (
        (worldometer_clean['TotalCases'] / worldometer_clean['TotalTests']) * 100
    ).round(2)
    
    # Replace inf with NaN
    worldometer_clean['Test_Positivity_Rate'] = worldometer_clean['Test_Positivity_Rate'].replace([np.inf, -np.inf], np.nan)
    
    available = worldometer_clean['Test_Positivity_Rate'].notna().sum()
    print(f"Test positivity rate calculated for {available} countries")
else:
    print("Cannot calculate - missing required columns")


# 4. Serious Cases Percentage

print("\n4. Calculating serious cases percentage...")

# Find the serious/critical column (it might have different names after cleaning)
serious_col = None
for col in worldometer_clean.columns:
    if 'serious' in col.lower() or 'critical' in col.lower():
        serious_col = col
        break

if serious_col and 'ActiveCases' in worldometer_clean.columns:
    worldometer_clean['Serious_Cases_Pct'] = (
        (worldometer_clean[serious_col] / worldometer_clean['ActiveCases']) * 100
    ).round(2)
    
    # Replace inf with NaN
    worldometer_clean['Serious_Cases_Pct'] = worldometer_clean['Serious_Cases_Pct'].replace([np.inf, -np.inf], np.nan)
    
    available = worldometer_clean['Serious_Cases_Pct'].notna().sum()
    print(f"Serious cases percentage calculated for {available} countries")
else:
    print("Cannot calculate - missing required columns")


# 5. Recovery Rate

print("\n5. Calculating recovery rate...")

if 'TotalRecovered' in worldometer_clean.columns and 'TotalCases' in worldometer_clean.columns:
    worldometer_clean['Recovery_Rate_Pct'] = (
        (worldometer_clean['TotalRecovered'] / worldometer_clean['TotalCases']) * 100
    ).round(2)
    
    worldometer_clean['Recovery_Rate_Pct'] = worldometer_clean['Recovery_Rate_Pct'].replace([np.inf, -np.inf], np.nan)
    
    print("Recovery rate calculated")
else:
    print("Cannot calculate - missing required columns")


# Summary

print("\nSummary of derived columns added:")
new_cols = [col for col in worldometer_clean.columns if '_Calculated' in col or '_Pct' in col or '_Rate' in col]
for col in new_cols:
    non_null = worldometer_clean[col].notna().sum()
    print(f"{col}: {non_null} countries have data")

print(f"\nTotal columns now: {len(worldometer_clean.columns)}")

print("\nDerived columns for worldometer_clean: COMPLETE!")


1. Calculating cases per million population...
Cases per million calculated

2. Calculating deaths per million population...
Deaths per million calculated

3. Calculating test positivity rate...
Test positivity rate calculated for 191 countries

4. Calculating serious cases percentage...
Serious cases percentage calculated for 118 countries

5. Calculating recovery rate...
Recovery rate calculated

Summary of derived columns added:
Cases_Per_Million_Calculated: 208 countries have data
Deaths_Per_Million_Calculated: 187 countries have data
Test_Positivity_Rate: 191 countries have data
Serious_Cases_Pct: 118 countries have data
Recovery_Rate_Pct: 205 countries have data

Total columns now: 21

Derived columns for worldometer_clean: COMPLETE!


<a id='summary'></a>
## 9. Final Summary

Overview of our cleaned and enhanced datasets ready for analysis.

In [20]:

# full_grouped_clean Summary

print("\nDATASET 1: full_grouped_clean")

print(f"Shape: {full_grouped_clean.shape[0]:,} rows × {full_grouped_clean.shape[1]} columns")
print(f"Date range: {full_grouped_clean['Date'].min().date()} to {full_grouped_clean['Date'].max().date()}")
print(f"Countries: {full_grouped_clean['Country/Region'].nunique()}")
print(f"WHO Regions: {full_grouped_clean['WHO Region'].nunique()}")

print(f"\nColumn Categories:")
print(f"Original columns: 10")
print(f"Derived columns: {len(full_grouped_clean.columns) - 10}")
print(f"Total columns: {len(full_grouped_clean.columns)}")

print(f"\nKey Statistics (Latest Date):")
latest_fg = full_grouped_clean[full_grouped_clean['Date'] == full_grouped_clean['Date'].max()]
print(f"Total Global Confirmed: {latest_fg['Confirmed'].sum():,}")
print(f"Total Global Deaths: {latest_fg['Deaths'].sum():,}")
print(f"Total Global Recovered: {latest_fg['Recovered'].sum():,}")

print(f"\nSample of derived columns:")
sample_derived = full_grouped_clean[full_grouped_clean['Country/Region'] == 'US'].tail(1)[
    ['Date', 'Country/Region', 'New cases', 'New_Cases_7MA', 'Mortality_Rate', 'Days_Since_First_Case']
]
display(sample_derived)


# worldometer_clean Summary

print("\nDATASET 2: worldometer_clean")

print(f"Shape: {worldometer_clean.shape[0]:,} rows × {worldometer_clean.shape[1]} columns")
print(f"Countries: {len(worldometer_clean)}")

if 'Continent' in worldometer_clean.columns:
    print(f"Continents: {worldometer_clean['Continent'].nunique()}")

print(f"\nData Availability:")
if 'Population' in worldometer_clean.columns:
    print(f"Population data: {worldometer_clean['Population'].notna().sum()} countries")
if 'TotalTests' in worldometer_clean.columns:
    print(f"Testing data: {worldometer_clean['TotalTests'].notna().sum()} countries")

print(f"\nTop 5 Countries by Cases Per Million:")
if 'Cases_Per_Million_Calculated' in worldometer_clean.columns:
    top5 = worldometer_clean.nlargest(5, 'Cases_Per_Million_Calculated')[
        ['Country_Region', 'Population', 'TotalCases', 'Cases_Per_Million_Calculated']
    ]
    display(top5)


# Memory Usage

print("\nMemory Usage:")
fg_memory = full_grouped_clean.memory_usage(deep=True).sum() / 1024**2
wm_memory = worldometer_clean.memory_usage(deep=True).sum() / 1024**2
print(f"full_grouped_clean: {fg_memory:.2f} MB")
print(f"worldometer_clean: {wm_memory:.2f} MB")
print(f"Total: {fg_memory + wm_memory:.2f} MB")

print("\nBoth datasets are clean, validated, and ready for analysis!")


DATASET 1: full_grouped_clean
Shape: 35,156 rows × 17 columns
Date range: 2020-01-22 to 2020-07-27
Countries: 187
WHO Regions: 6

Column Categories:
Original columns: 10
Derived columns: 7
Total columns: 17

Key Statistics (Latest Date):
Total Global Confirmed: 16,480,485
Total Global Deaths: 654,036
Total Global Recovered: 9,468,087

Sample of derived columns:


,Date,Country/Region,New cases,New_Cases_7MA,Mortality_Rate,Days_Since_First_Case
32711,2020-07-27,US,56336,65083.142857,3.45,187



DATASET 2: worldometer_clean
Shape: 209 rows × 21 columns
Countries: 209
Continents: 6

Data Availability:
Population data: 208 countries
Testing data: 191 countries

Top 5 Countries by Cases Per Million:


,Country_Region,Population,TotalCases,Cases_Per_Million_Calculated
24,Qatar,2807805.0,112092,39921.58
84,French Guiana,299385.0,8127,27145.65
47,Bahrain,1706669.0,42889,25130.24
157,San Marino,33938.0,699,20596.38
7,Chile,19132514.0,366671,19164.81



Memory Usage:
full_grouped_clean: 7.91 MB
worldometer_clean: 0.06 MB
Total: 7.97 MB

Both datasets are clean, validated, and ready for analysis!


<a id='save'></a>
## 10. Save Cleaned Data

Saving the cleaned datasets for use in subsequent analysis notebooks.

In [23]:
# Save full_grouped_clean

print("\n1. Saving full_grouped_clean...")
output_file1 = '../data/cleaned/full_grouped_clean.csv'
full_grouped_clean.to_csv(output_file1, index=False)




# Save worldometer_clean

print("\n2. Saving worldometer_clean...")
output_file2 = '../data/cleaned/worldometer_clean.csv'
worldometer_clean.to_csv(output_file2, index=False)



1. Saving full_grouped_clean...

2. Saving worldometer_clean...
